# Download Reconstruction Results 

Recursively download all results from EC2 instances.

In [1]:
import os
import paramiko
from stat import S_ISDIR
from tqdm import tqdm_notebook

In [2]:
BASE_REMOTE_PATH = "/home/ubuntu/code/pyshdom/scripts/results/camera_array_retrieval.py"
BASE_LOCAL_PATH =  "./results"
HOSTS = ("35.158.158.179", "35.158.233.59", "52.59.55.162")
DOWNLOAD_FILES = ('beta.pkl', 'params.txt')

In [3]:
def sftp_walk(remotepath):
    # Kindof a stripped down  version of os.walk, implemented for 
    # sftp.  Tried running it flat without the yields, but it really
    # chokes on big directories.
    path=remotepath
    files=[]
    folders=[]
    for f in ftp.listdir_attr(remotepath):
        if S_ISDIR(f.st_mode):
            folders.append(f.filename)
        else:
            files.append(f.filename)

    yield path, folders, files
    
    for folder in folders:
        new_path = '/'.join((remotepath, folder))
        for x in sftp_walk(new_path):
            yield x

In [4]:
k = paramiko.RSAKey.from_private_key_file(r"C:\Users\amitaid\Downloads\aviad_ec2.pem")
c = paramiko.SSHClient()
c.set_missing_host_key_policy(paramiko.AutoAddPolicy())
for host in tqdm_notebook(HOSTS):
    c.connect(hostname=host, username="ubuntu", pkey=k)
    ftp = c.open_sftp()
    for base_path, folder, files in tqdm_notebook(sftp_walk(BASE_REMOTE_PATH), leave=False):
        if 'beta.pkl' in files:
            for fname in DOWNLOAD_FILES:
                try:
                    src_path = '/'.join((base_path, fname))
                    dst_folder = os.path.join(BASE_LOCAL_PATH, base_path[len(BASE_REMOTE_PATH)+1:])

                    if not os.path.exists(dst_folder):
                        os.makedirs(dst_folder)

                    ftp.get(src_path, os.path.join(dst_folder, fname))
                except:
                    print("Failed copying {}".format(fname))
                    raise